In [0]:

from pyspark.sql.functions import *
from delta.tables import DeltaTable


In [0]:
%run /Workspace/consolidated_pipeline/1_setup/utilities

In [0]:
print(bronze_schema, silver_schema, gold_schema)

bronze silver gold


In [0]:
dbutils.widgets.text("catalog", "fmcg", "Catalog")
dbutils.widgets.text("data_source", "orders", "Data Source")

catalog = dbutils.widgets.get("catalog")
data_source = dbutils.widgets.get("data_source")

base_path = f's3://pranshu-sports-bar/{data_source}'
landing_path = f"{base_path}/landing/"
processed_path = f"{base_path}/processed/"
print("Base Path: ", base_path)
print("Landing Path: ", landing_path)
print("Processed Path: ", processed_path)


# define the tables
bronze_table = f"{catalog}.{bronze_schema}.{data_source}"
silver_table = f"{catalog}.{silver_schema}.{data_source}"
gold_table = f"{catalog}.{gold_schema}.sb_fact_{data_source}"

Base Path:  s3://pranshu-sports-bar/orders
Landing Path:  s3://pranshu-sports-bar/orders/landing/
Processed Path:  s3://pranshu-sports-bar/orders/processed/


## Bronze

In [0]:
df = spark.read.options(header=True, inferSchema=True) \
  .csv(f"{landing_path}/*.csv") \
    .withColumn("read_timestamp", current_timestamp()) \
      .select("*", "_metadata.file_name", "_metadata.file_size")

print("Total Rows: ", df.count())
df.show(5)

Total Rows:  51810
+------------+--------------------+-----------+----------+---------+--------------------+--------------------+---------+
|    order_id|order_placement_date|customer_id|product_id|order_qty|      read_timestamp|           file_name|file_size|
+------------+--------------------+-----------+----------+---------+--------------------+--------------------+---------+
|FOCT62720602|Tuesday, Septembe...|     ABC987|  25891301|     71.0|2026-01-19 21:30:...|orders_2025_09_30...|    41446|
|FOCT62720602|Tuesday, Septembe...|     789720|  25891502|    125.0|2026-01-19 21:30:...|orders_2025_09_30...|    41446|
|FOCT62720602|Tuesday, Septembe...|     789720|  25891403|    462.0|2026-01-19 21:30:...|orders_2025_09_30...|    41446|
|FOCT62720602|Tuesday, Septembe...|    INVALID|  25891601|    133.0|2026-01-19 21:30:...|orders_2025_09_30...|    41446|
|FOCT62720602|Tuesday, Septembe...|     789720|  25891602|     79.0|2026-01-19 21:30:...|orders_2025_09_30...|    41446|
+------------

In [0]:
display(df.limit(20))

order_id,order_placement_date,customer_id,product_id,order_qty,read_timestamp,file_name,file_size
FOCT62720602,"Tuesday, September 30, 2025",ABC987,25891301,71.0,2026-01-19T21:32:18.237Z,orders_2025_09_30.csv,41446
FOCT62720602,"Tuesday, September 30, 2025",789720,25891502,125.0,2026-01-19T21:32:18.237Z,orders_2025_09_30.csv,41446
FOCT62720602,"Tuesday, September 30, 2025",789720,25891403,462.0,2026-01-19T21:32:18.237Z,orders_2025_09_30.csv,41446
FOCT62720602,"Tuesday, September 30, 2025",INVALID,25891601,133.0,2026-01-19T21:32:18.237Z,orders_2025_09_30.csv,41446
FOCT62720602,"Tuesday, September 30, 2025",789720,25891602,79.0,2026-01-19T21:32:18.237Z,orders_2025_09_30.csv,41446
FOCT62720602,2025/09/30,XYZ123,25891101,472.0,2026-01-19T21:32:18.237Z,orders_2025_09_30.csv,41446
FOCT62720602,2025/09/30,ABC987,25891301,71.0,2026-01-19T21:32:18.237Z,orders_2025_09_30.csv,41446
FOCT62402303,"Tuesday, September 30, 2025",789402,25891303,24.0,2026-01-19T21:32:18.237Z,orders_2025_09_30.csv,41446
FOCT62402303,"Tuesday, September 30, 2025",789402,25891301,26.0,2026-01-19T21:32:18.237Z,orders_2025_09_30.csv,41446
FOCT62402303,"Tuesday, September 30, 2025",789402,25891102,328.0,2026-01-19T21:32:18.237Z,orders_2025_09_30.csv,41446


In [0]:
df.write\
 .format("delta") \
 .option("delta.enableChangeDataFeed", "true") \
 .mode("append") \
 .saveAsTable(bronze_table)

### Moving files from landing to processed

In [0]:
files = dbutils.fs.ls(landing_path) ## returns a list of FileInfo objects

for file_info in files:
    dbutils.fs.mv(
        file_info.path,
        f"{processed_path}/{file_info.name}",
        True
    )

## Silver

In [0]:
df_orders = spark.sql(f"SELECT * FROM {bronze_table}")
df_orders.show(2)

+------------+--------------------+-----------+----------+---------+--------------------+--------------------+---------+
|    order_id|order_placement_date|customer_id|product_id|order_qty|      read_timestamp|           file_name|file_size|
+------------+--------------------+-----------+----------+---------+--------------------+--------------------+---------+
|FJUL33320501|          2025/07/01|     789320|  25891203|    150.0|2026-01-19 21:33:...|orders_2025_07_01...|    20744|
|FJUL33320501|          2025/07/01|     789320|  25891301|     46.0|2026-01-19 21:33:...|orders_2025_07_01...|    20744|
+------------+--------------------+-----------+----------+---------+--------------------+--------------------+---------+
only showing top 2 rows


### Transformation

In [0]:
# 1. Keep only rows where order_qty is present
# we talj to business head and they agree to filter out those records which dont have any order_qty
df_orders = df_orders.filter(F.col("order_qty").isNotNull()) 

In [0]:
# 2. Clean customer_id → keep numeric, else set to 999999
df_orders = df_orders.withColumn(
    "customer_id",
    F.when(F.col("customer_id").rlike("^[0-9]+$"), F.col("customer_id"))
     .otherwise("999999")
     .cast("string")
)

In [0]:
# 3. Remove weekday name from the date text
#    "Tuesday, July 01, 2025" → "July 01, 2025"
df_orders = df_orders.withColumn(
    "order_placement_date",
    F.regexp_replace(F.col("order_placement_date"), r"^[A-Za-z]+,\s*", "")
)

In [0]:
# 4. Parse order_placement_date using multiple possible formats
df_orders = df_orders.withColumn(
    "order_placement_date",
    F.coalesce(
        F.try_to_date("order_placement_date", "yyyy/MM/dd"),
        F.try_to_date("order_placement_date", "dd-MM-yyyy"),
        F.try_to_date("order_placement_date", "dd/MM/yyyy"),
        F.try_to_date("order_placement_date", "MMMM dd, yyyy"),
    )
)

In [0]:
# 5. Drop duplicates
df_orders = df_orders.dropDuplicates(["order_id", "order_placement_date", "customer_id", "product_id", "order_qty"])

# 6. convert product id to string
df_orders = df_orders.withColumn('product_id', F.col('product_id').cast('string'))

In [0]:
# check what's the maximum and minimum date
df_orders.agg(
    F.min("order_placement_date").alias("min_date"),
    F.max("order_placement_date").alias("max_date")
).show()

+----------+----------+
|  min_date|  max_date|
+----------+----------+
|2025-07-01|2025-11-30|
+----------+----------+



In [0]:
display(df_orders.limit(20))

order_id,order_placement_date,customer_id,product_id,order_qty,read_timestamp,file_name,file_size
FJUL33320501,2025-07-01,789320,25891203,150.0,2026-01-19T21:33:57.728Z,orders_2025_07_01.csv,20744
FJUL33320501,2025-07-01,789320,25891301,46.0,2026-01-19T21:33:57.728Z,orders_2025_07_01.csv,20744
FJUL33320501,2025-07-01,789320,25891201,354.0,2026-01-19T21:33:57.728Z,orders_2025_07_01.csv,20744
FJUL33320501,2025-07-01,789320,25891501,249.0,2026-01-19T21:33:57.728Z,orders_2025_07_01.csv,20744
FJUL33401603,2025-07-01,789401,25891302,40.0,2026-01-19T21:33:57.728Z,orders_2025_07_01.csv,20744
FJUL33401603,2025-07-01,789401,25891502,133.0,2026-01-19T21:33:57.728Z,orders_2025_07_01.csv,20744
FJUL33401603,2025-07-01,789401,25891503,145.0,2026-01-19T21:33:57.728Z,orders_2025_07_01.csv,20744
FJUL33401603,2025-07-01,789401,25891203,429.0,2026-01-19T21:33:57.728Z,orders_2025_07_01.csv,20744
FJUL33401603,2025-07-01,789401,25891201,461.0,2026-01-19T21:33:57.728Z,orders_2025_07_01.csv,20744
FJUL32101601,2025-07-01,789101,25891503,183.0,2026-01-19T21:33:57.728Z,orders_2025_07_01.csv,20744


### join using products so that we can get product_code

In [0]:
df_products = spark.table("fmcg.silver.products")
df_joined = df_orders.join(df_products, on="product_id", how="inner").select(df_orders["*"], df_products["product_code"])

display(df_joined.limit(10))

order_id,order_placement_date,customer_id,product_id,order_qty,read_timestamp,file_name,file_size,product_code
FJUL312422401,2025-07-10,789422,25891401,406.0,2026-01-19T21:33:57.728Z,orders_2025_07_10.csv,20202,da6bfc596c1360ca07bda4e0ae6bfe3b8456517fc6e8ddc265630ff940f9ab05
FJUL316103602,2025-07-14,789103,25891403,235.0,2026-01-19T21:33:57.728Z,orders_2025_07_14.csv,20354,77b6f538a9d0e0cf845db5c2cbecec46fdd30303b501e06f64baf1d4dc0e66f9
FJUL316402601,2025-07-14,789402,25891601,167.0,2026-01-19T21:33:57.728Z,orders_2025_07_14.csv,20354,716fa4e54b7894c910180276e0535d49afb25cdcfac09533fb74ae00689e5742
FJUL320720201,2025-07-19,789720,25891103,358.0,2026-01-19T21:33:57.728Z,orders_2025_07_19.csv,21044,102628255d24304d6bbe0438b1ac992054f262e0814d306d0a34d7356cef3268
FJUL321403603,2025-07-19,789403,25891603,169.0,2026-01-19T21:33:57.728Z,orders_2025_07_19.csv,21044,451f7167b28a25bde73995910e31c07dfa26411f1db47847f19e16747effbdaa
FJUL328403503,2025-07-27,789403,25891101,367.0,2026-01-19T21:33:57.728Z,orders_2025_07_27.csv,20056,e91ba9d665f90254da5809bfdebe3db2be01a52f50b6fd96b57eed238392b843
FAUG46903403,2025-08-03,789903,25891302,59.0,2026-01-19T21:33:57.728Z,orders_2025_08_03.csv,20414,d9ebd1ca64d23951a6310af93b1c5ac27d831ac842e89aea59a9e8b38621faa5
FAUG46303602,2025-08-03,789303,25891303,85.0,2026-01-19T21:33:57.728Z,orders_2025_08_03.csv,20414,c68834ceaff15846bc1892c2185dc4e4f471d64fe3796b1a8ecc39a5a48c614f
FAUG411902303,2025-08-10,789902,25891103,436.0,2026-01-19T21:33:57.728Z,orders_2025_08_10.csv,19957,102628255d24304d6bbe0438b1ac992054f262e0814d306d0a34d7356cef3268
FAUG417402403,2025-08-14,789402,25891403,382.0,2026-01-19T21:33:57.728Z,orders_2025_08_14.csv,20782,77b6f538a9d0e0cf845db5c2cbecec46fdd30303b501e06f64baf1d4dc0e66f9


In [0]:
# so even in the full load data we still make sure that upsert code is using

if not (spark.catalog.tableExists(silver_table)):
    df_joined.write.format("delta").option(
        "delta.enableChangeDataFeed", "true"
    ).option("mergeSchema", "true").mode("overwrite").saveAsTable(silver_table)
else:
    silver_delta = DeltaTable.forName(spark, silver_table)
    silver_delta.alias("silver").merge(df_joined.alias("bronze"), "silver.order_placement_date = bronze.order_placement_date AND silver.order_id = bronze.order_id AND silver.product_code = bronze.product_code AND silver.customer_id = bronze.customer_id").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

## Gold

In [0]:
df_gold = spark.sql(f"SELECT order_id, order_placement_date as date, customer_id as customer_code, product_code, product_id, order_qty as sold_quantity FROM {silver_table};")

df_gold.show(2)

+------------+----------+-------------+--------------------+----------+-------------+
|    order_id|      date|customer_code|        product_code|product_id|sold_quantity|
+------------+----------+-------------+--------------------+----------+-------------+
|FOCT67703603|2025-10-06|       789703|778c2a7aa27bfdb21...|  25891602|         88.0|
|FOCT65220102|2025-10-04|       789220|e92c739a8d78cd6cb...|  25891102|        402.0|
+------------+----------+-------------+--------------------+----------+-------------+
only showing top 2 rows


In [0]:
# same upsert code

if not (spark.catalog.tableExists(gold_table)):
    print("creating New Table")
    df_gold.write.format("delta").option(
        "delta.enableChangeDataFeed", "true"
    ).option("mergeSchema", "true").mode("overwrite").saveAsTable(gold_table)
else:
    gold_delta = DeltaTable.forName(spark, gold_table)
    gold_delta.alias("source").merge(df_gold.alias("gold"), "source.date = gold.date AND source.order_id = gold.order_id AND source.product_code = gold.product_code AND source.customer_code = gold.customer_code").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

creating New Table


### Merging with parent data

In [0]:
df_child = spark.sql(f"SELECT date, product_code, customer_code, sold_quantity FROM {gold_table}")
df_child.show(10)

+----------+--------------------+-------------+-------------+
|      date|        product_code|customer_code|sold_quantity|
+----------+--------------------+-------------+-------------+
|2025-10-06|778c2a7aa27bfdb21...|       789703|         88.0|
|2025-10-04|e92c739a8d78cd6cb...|       789220|        402.0|
|2025-08-10|77b6f538a9d0e0cf8...|       789103|        324.0|
|2025-10-25|0cb7b2f42657b625f...|       999999|        337.0|
|2025-11-21|451f7167b28a25bde...|       789601|        105.0|
|2025-07-11|716fa4e54b7894c91...|       789421|        162.0|
|2025-09-07|ee1f7df9cf660ef02...|       789401|        182.0|
|2025-10-11|889c67757ece9c973...|       789902|        273.0|
|2025-11-26|2e387cef1424d6e7b...|       789903|        193.0|
|2025-10-17|da6bfc596c1360ca0...|       789103|        352.0|
+----------+--------------------+-------------+-------------+
only showing top 10 rows


In [0]:
df_child.count()

40811

In [0]:
df_monthly = (
    df_child
    # 1. Get month start date (e.g., 2025-11-30 → 2025-11-01)
    .withColumn("month_start", trunc("date", "MM"))   # or F.date_trunc("month", "date").cast("date")

    # 2.Group at monthly grain by month_start + product_code + customer_code
    .groupBy("month_start", "product_code", "customer_code")
    .agg(
        sum("sold_quantity").alias("sold_quantity")
    )

    # 3. Rename month_start back to `date` to match your target schema
    .withColumnRenamed("month_start", "date")
)

df_monthly.show(5, truncate=False)

+----------+----------------------------------------------------------------+-------------+-------------+
|date      |product_code                                                    |customer_code|sold_quantity|
+----------+----------------------------------------------------------------+-------------+-------------+
|2025-10-01|778c2a7aa27bfdb211fd5ece048de80d00fbf3d6924bd908d91054796ba16ab6|789703       |1253.0       |
|2025-10-01|e92c739a8d78cd6cbe954648c2f9dd75ed61fcfd99b03e10dca65c3082d0728e|789220       |6045.0       |
|2025-08-01|77b6f538a9d0e0cf845db5c2cbecec46fdd30303b501e06f64baf1d4dc0e66f9|789103       |4437.0       |
|2025-10-01|0cb7b2f42657b625f754e833aa1cf6a967be26f17415f5342302ebb0e90c8a28|999999       |15586.0      |
|2025-11-01|451f7167b28a25bde73995910e31c07dfa26411f1db47847f19e16747effbdaa|789601       |1962.0       |
+----------+----------------------------------------------------------------+-------------+-------------+
only showing top 5 rows


In [0]:
df_monthly.count()

3060

In [0]:
gold_parent_delta = DeltaTable.forName(spark, f"{catalog}.{gold_schema}.fact_orders")
gold_parent_delta.alias("parent_gold").merge(df_monthly.alias("child_gold"), "parent_gold.date = child_gold.date AND parent_gold.product_code = child_gold.product_code AND parent_gold.customer_code = child_gold.customer_code").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]